In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime, Date, Time
from sqlalchemy.orm import Session

In [3]:
import datetime as dt

In [4]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [5]:
# Creates Classes which will serve as the anchor points for our Table
class SP_Data(Base):
    __tablename__ = 'sp_data_20200320'
    id = Column(Integer, primary_key=True)
    date = Column(Date)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    adj_close = Column(Float)
    volume = Column(Integer)

In [6]:
# Create Database Connection
engine = create_engine("sqlite:///../../db/bulls_bears.sqlite")
conn = engine.connect()

In [7]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [8]:
# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)
# Base.metadata.clear()

In [9]:
from sqlalchemy import inspect

In [10]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [11]:
# Collect the names of tables within the database
inspector.get_table_names()

['sp_data', 'sp_data_20200320']

In [12]:
# Using the inspector to print the column names within the 'tweet_data' table and its types
columns = inspector.get_columns('sp_data_20200320')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
date DATE
open FLOAT
high FLOAT
low FLOAT
close FLOAT
adj_close FLOAT
volume INTEGER


In [13]:
data_file = "../../datasets/SAP500_20200320.csv"

In [14]:
sp_df = pd.read_csv(data_file)

In [15]:
sp_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
1,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2,2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
3,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
4,2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [16]:
sp_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [19]:
sp_df["Date"] = sp_df["Date"].apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").date())

In [20]:
sp_df = sp_df.rename(columns={"Date": "date", "Open": "open", "High": "high",
                              "Low": "low", "Close": "close",
                              "Adj Close": "adj_close", "Volume": "volume"})

In [21]:
sp_df.head()

,date,open,high,low,close,adj_close,volume
0,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
1,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2,2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
3,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
4,2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [22]:
#Export data to sqlite
sp_df.to_sql('sp_data_20200320', con=engine, if_exists = 'append', index = False)

In [23]:
engine.execute("SELECT * FROM sp_data LIMIT 15").fetchall()

[(1, '2007-01-03', 1418.030029, 1429.420044, 1407.859985, 1416.599976, 1416.599976, 3429160000),
 (2, '2007-01-04', 1416.599976, 1421.839966, 1408.430054, 1418.339966, 1418.339966, 3004460000),
 (3, '2007-01-05', 1418.339966, 1418.339966, 1405.75, 1409.709961, 1409.709961, 2919400000),
 (4, '2007-01-08', 1409.26001, 1414.97998, 1403.969971, 1412.839966, 1412.839966, 2763340000),
 (5, '2007-01-09', 1412.839966, 1415.609985, 1405.420044, 1412.109985, 1412.109985, 3038380000),
 (6, '2007-01-10', 1408.699951, 1415.98999, 1405.319946, 1414.849976, 1414.849976, 2764660000),
 (7, '2007-01-11', 1414.839966, 1427.119995, 1414.839966, 1423.819946, 1423.819946, 2857870000),
 (8, '2007-01-12', 1423.819946, 1431.22998, 1422.579956, 1430.72998, 1430.72998, 2686480000),
 (9, '2007-01-16', 1430.72998, 1433.930054, 1428.619995, 1431.900024, 1431.900024, 2599530000),
 (10, '2007-01-17', 1431.77002, 1435.27002, 1428.569946, 1430.619995, 1430.619995, 2690270000),
 (11, '2007-01-18', 1430.589966, 1432.9599

In [27]:
pd.read_sql("SELECT * FROM sp_data_20200320 ORDER BY date DESC LIMIT 15", con = conn).reset_index(drop = True)

,id,date,open,high,low,close,adj_close,volume
0,5085,2020-03-19,2393.479980,2466.969971,2319.780029,2409.389893,2409.389893,7946710000
1,5084,2020-03-18,2436.500000,2453.570068,2280.520020,2398.100098,2398.100098,8755780000
2,5083,2020-03-17,2425.659912,2553.929932,2367.040039,2529.189941,2529.189941,8358500000
3,5082,2020-03-16,2508.590088,2562.979980,2380.939941,2386.129883,2386.129883,7781540000
4,5081,2020-03-13,2569.989990,2711.330078,2492.370117,2711.020020,2711.020020,8258670000
5,5080,2020-03-12,2630.860107,2660.949951,2478.860107,2480.639893,2480.639893,8829380000
6,5079,2020-03-11,2825.600098,2825.600098,2707.219971,2741.379883,2741.379883,7374110000
7,5078,2020-03-10,2813.479980,2882.590088,2734.000000,2882.229980,2882.229980,7635960000
8,5077,2020-03-09,2863.889893,2863.889893,2734.429932,2746.560059,2746.560059,8423050000
9,5076,2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000


In [25]:
session = Session(engine)

In [26]:
session.query(func.max(SP_Data.date)).all()

[(datetime.date(2020, 3, 19))]

In [100]:
session.query(func.min(SP_Data.date)).all()

[(datetime.date(2007, 1, 3))]